# **World Port Index Analysis**

## **Introduction**

The inspiration for the world's best port and terminal data analysis project came from wanting to venture past my comfort zone and gain insights into the backbone of logistics.

This data analysis project's primary purpose is to pinpoint which major ports and terminals in the world excel above all those included in the dataset. The metrics, i.e., data fields, will be used to thoroughly reveal which ports have claimed top positions in terms of the various categories present in the dataset, which include location, physical characteristics, available facilities, and services offered. The analysis will also provide valuable insights for anyone interested in the maritime industry.

The dataset was sourced from the National Geospatial-Intelligence Agency, a federal agency in the United States. The agency validates, updates, and maintains the data. The data is updated monthly.

Link:  https://msi.nga.mill/Publications/WPI
<br>   21/11/2022


#### Dataset sources
Explanation of data fields
<br> Link: https://msi.nga.mil/api/publications/download?key=16920959/SFH00000/WPI_Explanation_of_Data_Fields.pdf&type=view
<br> Dataset
<br> Link: https://msi.nga.mil/api/publications/download?type=view&key=16920959/SFH00000/UpdatedPub150.csv

In [3]:
from urllib import request
import camelot
import glob
import re
import zipfile
import pandas as pd

In [7]:
# This cell will download the dataset and explanation of the data fields to the current directory after providing the links above in third cell.
def get_data(URL):
    """ Download files from official website.  """
   
    if file_name := re.search("^https://msi\.nga\.mil/api/publications/download\?(?:.+)/SFH00000/(.+\.(csv|pdf))(.+)?", URL, re.IGNORECASE):
        request.urlretrieve(URL, file_name.group(1))
        print("Download successful")
    else:
        print("Error has occured")

get_data(input("Url: "))

Download successful


#### Generate a schema
To avoid going back and forth between different applications, a schema to explain the data fields is generated for use.

In [4]:
def extract_tables(pdf_name):
    """
    Extract tables from WPI_Explanation_of_Data_Fields and compress them
    pdf_name: name of pdf to extract tables
    """
    explanation_tables = camelot.read_pdf(pdf_name, pages= "1, 2-9")
    explanation_tables.export("raw_schema.csv", f= "csv", compress= True)

    with zipfile.ZipFile("raw_schema.zip", "r") as zip_ref:
        return zip_ref.extractall("raw_schema")
    
extract_tables("WPI_Explanation_of_Data_Fields.pdf")

In [5]:
def merge_csv_files(path):
    """
    Merge all csv files into one from extracted tables from the pdf
    path: path of unzipped folder
    """
    merged_exp_tables = pd.concat(map(pd.read_csv, glob.glob(path + "/*.csv")), ignore_index= True)
    return merged_exp_tables.to_csv("preschema.csv", index= False)

 #path has to have double back slashes to work
merge_csv_files("C:\\Users\\DSL\\Documents\\python\\WPA\\raw_schema")

In [6]:
schema_df = pd.read_csv("preschema.csv", header= 0, names= ["Field_Name", "Description_of_Contents"])
schema_df.replace(r"\r+|\n+|\t+", "", regex= True, inplace= True)
schema_df

,Field_Name,Description_of_Contents
0,OID [OID],A common identifier is given for all entries i...
1,WPI Number [wpinumber],Each port and place contained in this dataset ...
2,Region Name [regionname],"Region Names, as given in the 2019 edition of ..."
3,Main Port Name [main_port_],"Main Port Name, with no diacritical marks. In..."
4,Alternate Port Name [alternate_],"Alternate, conventional, or familiar names may..."
...,...,...
104,Repairs [repair_cod],Repairs that can be made to ocean-going vessel...
105,Dry Dock [dry_dock],The general size and type of the largest under...
106,Railway [railway],The general size and type of the largest under...
107,Latitude [Latitude],Latitude of the port is given.


In [7]:
def clean_field_names(df):
    """
    Remove extra words from Field_Name in the schema in order to match them with data fields in the data set.
    df: Pandas data frame
    NB: Function can only be called after reading preschema csv with pandas
    """
    field_list = df["Field_Name"].tolist() # Convert specified column to a list
    modified_field_names = []
    try:
        for values in field_list: 
            if match := re.search("(.+)\[.+\]", values):
                modified_field_names.append(match.group(1).strip())

        #Replace current column with new generated column
        df["Field_Name"] = df["Field_Name"].replace(field_list, modified_field_names)
        return df

    except ValueError:
        print("Function can only be called after reading preschema csv as pandas dataframe as Field_name column has already been cleaned before.")

clean_field_names(schema_df)
schema_df.to_csv("schema.csv", index= False)

In [8]:
# Schema to refer to for the explanation of data fields in the dataset
schema_df = pd.read_csv("schema.csv", index_col= "Field_Name").Description_of_Contents
schema_df

Field_Name
OID                    A common identifier is given for all entries i...
WPI Number             Each port and place contained in this dataset ...
Region Name            Region Names, as given in the 2019 edition of ...
Main Port Name         Main Port Name, with no diacritical marks.  In...
Alternate Port Name    Alternate, conventional, or familiar names may...
                                             ...                        
Repairs                Repairs that can be made to ocean-going vessel...
Dry Dock               The general size and type of the largest under...
Railway                The general size and type of the largest under...
Latitude                                  Latitude of the port is given.
Longitude                                Longitude of the port is given.
Name: Description_of_Contents, Length: 109, dtype: object

In [9]:
schema_df["Channel Depth (m)"]

'The controlling depth of the principal or deepest channel at chart datum is given.  The channel selected should lead up to the anchorage if within the harbor, or to the wharf/pier.  If the channel depth decreases from the anchorage to the wharf/pier and cargo can be worked at the anchorage, then the depth leading to the anchorage is given.'

## **Data Wrangling**

The dataset, UpdatedPub150.csv, consists of 3739 rows and 108 columns. Some columns will be discarded as they will not aid in any part of the analysis, leaving 90 columns. The dataset is clean, with no errors or duplicate records, and all formats have been standardized.

In [10]:
world_ports_raw_df = pd.read_csv("UpdatedPub150.csv")
world_ports_raw_df

,World Port Index Number,Region Name,Main Port Name,Alternate Port Name,UN/LOCODE,Country Code,World Water Body,IHO S-130 Sea Area,Sailing Direction or Publication,Publication Link,...,Supplies - Fuel Oil,Supplies - Diesel Oil,Supplies - Aviation Fuel,Supplies - Deck,Supplies - Engine,Repairs,Dry Dock,Railway,Latitude,Longitude
0,7950.0,United States E Coast -- 6585,Maurer,,,United States,North Atlantic Ocean,,U.S. Coast Pilot 2 - Atlantic Coast: Cape Cod ...,https://nauticalcharts.noaa.gov/publications/c...,...,Yes,Yes,Unknown,Yes,Yes,Moderate,Unknown,Unknown,40.533333,-74.250000
1,52235.0,Sulawesi -- 51970,Mangkasa Oil Terminal,,,Indonesia,Teluk Bone; Banda Sea; South Pacific Ocean,,Sailing Directions Pub. 163 (Enroute) - Borneo...,https://msi.geo.nga.mil/api/publications/downl...,...,No,No,Unknown,No,No,None,Unknown,Unknown,-2.733333,121.066667
2,47620.0,Madagascar -- 47350,Iharana,,,Madagascar,Indian Ocean,,Sailing Directions Pub. 171 (Enroute) - East A...,https://msi.geo.nga.mil/api/publications/downl...,...,No,No,Unknown,No,No,Emergency Only,Unknown,Unknown,-13.350000,50.000000
3,47360.0,Madagascar -- 47350,Andoany,,,Madagascar,Mozambique Channel; Indian Ocean,,Sailing Directions Pub. 171 (Enroute) - East A...,https://msi.geo.nga.mil/api/publications/downl...,...,No,No,Unknown,No,No,Emergency Only,Unknown,Unknown,-13.400000,48.300000
4,47020.0,Tanzania -- 46965,Chake Chake,,,Tanzania,Indian Ocean,,Sailing Directions Pub. 171 (Enroute) - East A...,https://msi.geo.nga.mil/api/publications/downl...,...,No,No,Unknown,No,No,Unknown,Unknown,Unknown,-5.250000,39.766667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3734,1560.0,Newfoundland E Coast -- 1410,Lewisporte,,CA LWP,Canada,North Atlantic Ocean,,Sailing Directions Pub. 146 (Enroute) - Newfou...,https://msi.geo.nga.mil/api/publications/downl...,...,Yes,Yes,Unknown,Unknown,Unknown,Limited,Small,Unknown,49.250000,-55.050000
3735,1150.0,,Deception Bay,,CA DEB,Canada,Hudson Strait; Arctic Ocean,,Sailing Directions Pub. 146 (Enroute) - Newfou...,https://msi.geo.nga.mil/api/publications/downl...,...,Yes,Yes,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,62.150000,-74.683333
3736,1260.0,,Nain,,CA NAI,Canada,Labrador Sea; North Atlantic Ocean,,Sailing Directions Pub. 146 (Enroute) - Newfou...,https://msi.geo.nga.mil/api/publications/downl...,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,56.550000,-61.683333
3737,1540.0,Newfoundland E Coast -- 1410,Botwood,,CA BWD,Canada,North Atlantic Ocean,,Sailing Directions Pub. 146 (Enroute) - Newfou...,https://msi.geo.nga.mil/api/publications/downl...,...,Yes,Yes,Unknown,Unknown,Unknown,Limited,Unknown,Unknown,49.133333,-55.333333


In [11]:
# Returns all data field names
world_ports_raw_df.columns.values

array(['World Port Index Number', 'Region Name', 'Main Port Name',
       'Alternate Port Name', 'UN/LOCODE', 'Country Code',
       'World Water Body', 'IHO S-130 Sea Area',
       'Sailing Direction or Publication', 'Publication Link',
       'Standard Nautical Chart',
       'IHO S-57 Electronic Navigational Chart',
       'IHO S-101 Electronic Navigational Chart',
       'Digital Nautical Chart', 'Tidal Range (m)', 'Entrance Width (m)',
       'Channel Depth (m)', 'Anchorage Depth (m)', 'Cargo Pier Depth (m)',
       'Oil Terminal Depth (m)',
       'Liquified Natural Gas Terminal Depth (m)',
       'Maximum Vessel Length (m)', 'Maximum Vessel Beam (m)',
       'Maximum Vessel Draft (m)', 'Offshore Maximum Vessel Length (m)',
       'Offshore Maximum Vessel Beam (m)',
       'Offshore Maximum Vessel Draft (m)', 'Harbor Size', 'Harbor Type',
       'Harbor Use', 'Shelter Afforded', 'Entrance Restriction - Tide',
       'Entrance Restriction - Heavy Swell', 'Entrance Restriction - Ic

In [12]:
selected_columns = [
# General Port information
"World Port Index Number",
"Region Name",
"Main Port Name",
"Alternate Port Name",
"UN/LOCODE",
"Country Code",
"World Water Body",
"IHO S-130 Sea Area",
'Entrance Width (m)',
'Channel Depth (m)', 
'Anchorage Depth (m)',
'Cargo Pier Depth (m)',
'Oil Terminal Depth (m)',
'Maximum Vessel Length (m)', 
'Maximum Vessel Beam (m)',
'Maximum Vessel Draft (m)',
'Offshore Maximum Vessel Length (m)',
'Offshore Maximum Vessel Beam (m)',
'Offshore Maximum Vessel Draft (m)',
'Harbor Size', 
'Harbor Type',
'Harbor Use',
# Limitations present (values; Yes, No, Unknown)
'Entrance Restriction - Tide',
'Entrance Restriction - Heavy Swell', 
'Entrance Restriction - Ice',
'Entrance Restriction - Other',
'Overhead Limits',
# Systems in place
'Good Holding Ground',
'Turning Area',
'Shelter Afforded',
'Port Security',
'Estimated Time of Arrival Message', 
'Quarantine - Pratique',
'Quarantine - Sanitation', 
'Quarantine - Other',
'Traffic Separation Scheme', 
'Vessel Traffic Service',
# Critical infrastructure avaiilable (values; Yes, No, Unknown)
'Tugs - Salvage', 
'Tugs - Assistance',
'Communications - Telephone', 
'Communications - Telefax',
'Communications - Radio', 
'Communications - Radiotelephone',
'Communications - Airport', 
'Communications - Rail',
'Search and Rescue',
# Facilities available (values; Yes, No, Unknown)
'Facilities - Wharves',
'Facilities - Anchorage', 
'Facilities - Dangerous Cargo Anchorage',
'Facilities - Med Mooring', 
'Facilities - Beach Mooring',
'Facilities - Ice Mooring', 
'Facilities - Ro-Ro',
'Facilities - Solid Bulk', 
'Facilities - Liquid Bulk',
'Facilities - Container', 
'Facilities - Breakbulk',
'Facilities - Oil Terminal', 
'Facilities - LNG Terminal',
'Facilities - Other',
'Medical Facilities', 
'Garbage Disposal',
'Chemical Holding Tank Disposal', 
'Degaussing',
'Dirty Ballast Disposal',
# Equipment availability (values; Yes, No, Unknown)
'Cranes - Fixed', 
'Cranes - Mobile',
'Cranes - Floating', 
'Cranes - Container', 
'Lifts - 100+ Tons',
'Lifts - 50-100 Tons', 
'Lifts - 25-49 Tons', 
'Lifts - 0-24 Tons',
# Complimentary services availabity (values; Yes, No, Unknown)
'Services - Longshoremen', 
'Services - Electricity',
'Services - Steam', 
'Services - Navigation Equipment',
'Services - Electrical Repair', 
'Services - Ice Breaking',
'Services - Diving',
# General supplies availabilty (values; Yes, No, Unknown)
'Supplies - Provisions',
'Supplies - Potable Water', 
'Supplies - Fuel Oil',
'Supplies - Diesel Oil', 
'Supplies - Aviation Fuel',
'Supplies - Deck', 
'Supplies - Engine',
# Repair services
'Repairs',
'Dry Dock',
'Railway'
]

len(selected_columns)

90

In [13]:
# Generate a copy of the dataframe
preworld_ports_df = world_ports_raw_df[selected_columns].copy() 
preworld_ports_df

,World Port Index Number,Region Name,Main Port Name,Alternate Port Name,UN/LOCODE,Country Code,World Water Body,IHO S-130 Sea Area,Entrance Width (m),Channel Depth (m),...,Supplies - Provisions,Supplies - Potable Water,Supplies - Fuel Oil,Supplies - Diesel Oil,Supplies - Aviation Fuel,Supplies - Deck,Supplies - Engine,Repairs,Dry Dock,Railway
0,7950.0,United States E Coast -- 6585,Maurer,,,United States,North Atlantic Ocean,,0.0,11.0,...,Yes,Yes,Yes,Yes,Unknown,Yes,Yes,Moderate,Unknown,Unknown
1,52235.0,Sulawesi -- 51970,Mangkasa Oil Terminal,,,Indonesia,Teluk Bone; Banda Sea; South Pacific Ocean,,0.0,9.4,...,No,No,No,No,Unknown,No,No,None,Unknown,Unknown
2,47620.0,Madagascar -- 47350,Iharana,,,Madagascar,Indian Ocean,,0.0,14.0,...,Yes,Yes,No,No,Unknown,No,No,Emergency Only,Unknown,Unknown
3,47360.0,Madagascar -- 47350,Andoany,,,Madagascar,Mozambique Channel; Indian Ocean,,0.0,20.1,...,Unknown,Yes,No,No,Unknown,No,No,Emergency Only,Unknown,Unknown
4,47020.0,Tanzania -- 46965,Chake Chake,,,Tanzania,Indian Ocean,,0.0,14.0,...,No,No,No,No,Unknown,No,No,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3734,1560.0,Newfoundland E Coast -- 1410,Lewisporte,,CA LWP,Canada,North Atlantic Ocean,,0.0,0.0,...,Yes,Yes,Yes,Yes,Unknown,Unknown,Unknown,Limited,Small,Unknown
3735,1150.0,,Deception Bay,,CA DEB,Canada,Hudson Strait; Arctic Ocean,,0.0,0.0,...,Unknown,Yes,Yes,Yes,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
3736,1260.0,,Nain,,CA NAI,Canada,Labrador Sea; North Atlantic Ocean,,0.0,0.0,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
3737,1540.0,Newfoundland E Coast -- 1410,Botwood,,CA BWD,Canada,North Atlantic Ocean,,0.0,0.0,...,Yes,Yes,Yes,Yes,Unknown,Unknown,Unknown,Limited,Unknown,Unknown


### **Data Integration**

To answer the posed questions, a new dataset had to be introduced to aid in the analysis by merging them.

In [14]:
countries_continents_df = pd.read_csv("Countries-Continents.txt")
countries_continents_df

,Continent,Country
0,Africa,Algeria
1,Africa,Angola
2,Africa,Benin
3,Africa,Botswana
4,Africa,Burkina
...,...,...
189,South America,Paraguay
190,South America,Peru
191,South America,Suriname
192,South America,Uruguay


In [15]:
# Merging the two datasets 
world_ports_df = pd.merge(preworld_ports_df, countries_continents_df, 
                        left_on="Country Code",
                        right_on = "Country",
                        how = "left")
world_ports_df

,World Port Index Number,Region Name,Main Port Name,Alternate Port Name,UN/LOCODE,Country Code,World Water Body,IHO S-130 Sea Area,Entrance Width (m),Channel Depth (m),...,Supplies - Fuel Oil,Supplies - Diesel Oil,Supplies - Aviation Fuel,Supplies - Deck,Supplies - Engine,Repairs,Dry Dock,Railway,Continent,Country
0,7950.0,United States E Coast -- 6585,Maurer,,,United States,North Atlantic Ocean,,0.0,11.0,...,Yes,Yes,Unknown,Yes,Yes,Moderate,Unknown,Unknown,NaN,NaN
1,52235.0,Sulawesi -- 51970,Mangkasa Oil Terminal,,,Indonesia,Teluk Bone; Banda Sea; South Pacific Ocean,,0.0,9.4,...,No,No,Unknown,No,No,None,Unknown,Unknown,Asia,Indonesia
2,47620.0,Madagascar -- 47350,Iharana,,,Madagascar,Indian Ocean,,0.0,14.0,...,No,No,Unknown,No,No,Emergency Only,Unknown,Unknown,Africa,Madagascar
3,47360.0,Madagascar -- 47350,Andoany,,,Madagascar,Mozambique Channel; Indian Ocean,,0.0,20.1,...,No,No,Unknown,No,No,Emergency Only,Unknown,Unknown,Africa,Madagascar
4,47020.0,Tanzania -- 46965,Chake Chake,,,Tanzania,Indian Ocean,,0.0,14.0,...,No,No,Unknown,No,No,Unknown,Unknown,Unknown,Africa,Tanzania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3734,1560.0,Newfoundland E Coast -- 1410,Lewisporte,,CA LWP,Canada,North Atlantic Ocean,,0.0,0.0,...,Yes,Yes,Unknown,Unknown,Unknown,Limited,Small,Unknown,North America,Canada
3735,1150.0,,Deception Bay,,CA DEB,Canada,Hudson Strait; Arctic Ocean,,0.0,0.0,...,Yes,Yes,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,North America,Canada
3736,1260.0,,Nain,,CA NAI,Canada,Labrador Sea; North Atlantic Ocean,,0.0,0.0,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,North America,Canada
3737,1540.0,Newfoundland E Coast -- 1410,Botwood,,CA BWD,Canada,North Atlantic Ocean,,0.0,0.0,...,Yes,Yes,Unknown,Unknown,Unknown,Limited,Unknown,Unknown,North America,Canada


In [16]:
world_ports_df.to_csv("Major Ports & Terminals Analysis.csv", index= False)